# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Mod 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging than the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at a minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

## Import Necessary Libraries

In [1]:
import sqlite3 
import pandas as pd
import requests
import json
from datetime import timezone
import numpy as np
import pymongo
import matplotlib.pyplot as plt
%matplotlib inline

## Import SQL Database and Examine Data

In [2]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [3]:
cur.execute("""SELECT * FROM Matches WHERE Season = 2011;""")
df_Matches = pd.DataFrame(cur.fetchall())
df_Matches.columns = [x[0] for x in cur.description]
df_Matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


### Convert and Store Timestamp Values for Date

In [4]:
df_Matches['Date'] = pd.to_datetime(df_Matches['Date'])
df_Matches['Date'][0]

Timestamp('2012-03-31 00:00:00')

In [5]:
df_Matches['Timestamp'] = np.nan
for y in range(len(df_Matches)):
    df_Matches.loc[[y],'Timestamp'] = (df_Matches['Date'].iloc[y].replace(tzinfo=timezone.utc).timestamp())

In [6]:
cur.execute("""SELECT * FROM Teams;""")
df_Teams = pd.DataFrame(cur.fetchall())
df_Teams.columns = [x[0] for x in cur.description]
df_Teams.head()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [7]:
cur.execute("""SELECT * FROM Teams_in_Matches;""")
df_Teams_in_Matches = pd.DataFrame(cur.fetchall())
df_Teams_in_Matches.columns = [x[0] for x in cur.description]
df_Teams_in_Matches.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [8]:
cur.execute("""SELECT * FROM Unique_Teams;""")
df_Unique_Teams = pd.DataFrame(cur.fetchall())
df_Unique_Teams.columns = [x[0] for x in cur.description]
df_Unique_Teams.head()

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


## Call API and Store Weather for Each Match

In [129]:
df_Matches['Weather'] = np.nan
for y in range(len(df_Matches)):
    base_url = 'https://api.darksky.net/forecast/47e5f668133577877bde262eb2950de3/52.5200,13.4050,'
    daily_url = base_url + str(df_Matches['Timestamp'].iloc[y])
    daily_url = daily_url[:-2]
    response = requests.get(daily_url)
    match_json = response.json()
    match_json.keys()
    df_Matches.loc[[y],'Weather'] = match_json['daily']['data'][0]['icon']
df_Matches.head()

KeyError: 'daily'

## Select and Combine Useful Features

In [9]:
cur.execute("""SELECT HomeTeam, SUM(FTHG) AS HomeGoals
                FROM Matches
                WHERE Season = 2011
                GROUP BY HomeTeam;""")
df1 = pd.DataFrame(cur.fetchall())
df1.columns = [x[0] for x in cur.description]
df1.head()

,HomeTeam,HomeGoals
0,Aachen,15
1,Arsenal,39
2,Aston Villa,20
3,Augsburg,20
4,Bayern Munich,49


In [10]:
cur.execute("""SELECT AwayTeam, SUM(FTAG) AS AwayGoals
                FROM Matches
                WHERE Season = 2011
                GROUP BY AwayTeam;""")
df2 = pd.DataFrame(cur.fetchall())
df2.columns = [x[0] for x in cur.description]
df2.head()

,AwayTeam,AwayGoals
0,Aachen,15
1,Arsenal,35
2,Aston Villa,17
3,Augsburg,16
4,Bayern Munich,28


In [11]:
df = df1.merge(df2, how='inner', left_on='HomeTeam', right_on='AwayTeam')
df['TotalGoals'] = df['HomeGoals'] + df['AwayGoals']
df = df[['HomeTeam', 'TotalGoals']]
df.rename(columns={'HomeTeam':'TeamName'}, inplace = True)
df.head()

,TeamName,TotalGoals
0,Aachen,30
1,Arsenal,74
2,Aston Villa,37
3,Augsburg,36
4,Bayern Munich,77


In [12]:
df['Wins'] = 0
df['Losses'] = 0
df['Draws'] = 0
df['Total_Games'] = 0
df['Wins_in_Rain'] = 0
df['Games_in_Rain'] = 0

In [13]:
df.head()

,TeamName,TotalGoals,Wins,Losses,Draws,Total_Games,Wins_in_Rain,Games_in_Rain
0,Aachen,30,0,0,0,0,0,0
1,Arsenal,74,0,0,0,0,0,0
2,Aston Villa,37,0,0,0,0,0,0
3,Augsburg,36,0,0,0,0,0,0
4,Bayern Munich,77,0,0,0,0,0,0


In [19]:
for y in range(len(df_Matches)):
    idxh = df[df['TeamName'] == df_Matches.iloc[y]['HomeTeam']].index.tolist()
    idxa = df[df['TeamName'] == df_Matches.iloc[y]['AwayTeam']].index.tolist()
    if df_Matches.iloc[y]['FTR'] == 'H':
        df.loc[idxh,'Wins'] = df['Wins'][idxh] + 1
        df.loc[idxh,'Total_Games'] = df['Total_Games'][idxh] + 1
        df.loc[idxa,'Losses'] = df['Losses'][idxa] + 1
        df.loc[idxa,'Total_Games'] = df['Total_Games'][idxa] + 1 
#        if df_Matches.iloc[y]['Weather'] == 'rain':
#            df.loc[idxh,'Wins_in_Rain'] = df['Wins_in_Rain'][idxh] + 1
#            df.loc[idxh,'Games_in_Rain'] = df['Games_in_Rain'][idxh] + 1
#            df.loc[idxa,'Games_in_Rain'] = df['Games_in_Rain'][idxa] + 1 
    elif df_Matches.iloc[y]['FTR'] == 'D':
        df.loc[idxh,'Draws'] = df['Draws'][idxh] + 1
        df.loc[idxh,'Total_Games'] = df['Total_Games'][idxh] + 1
        df.loc[idxa,'Draws'] = df['Draws'][idxa] + 1
        df.loc[idxa,'Total_Games'] = df['Total_Games'][idxa] + 1
#        if df_Matches.iloc[y]['Weather'] == 'rain':
#            df.loc[idxh,'Games_in_Rain'] = df['Games_in_Rain'][idxh] + 1
#            df.loc[idxa,'Games_in_Rain'] = df['Games_in_Rain'][idxa] + 1 
    else:
        df.loc[idxh,'Losses'] = df['Losses'][idxh] + 1
        df.loc[idxh,'Total_Games'] = df['Total_Games'][idxh] + 1
        df.loc[idxa,'Wins'] = df['Wins'][idxa] + 1
        df.loc[idxa,'Total_Games'] = df['Total_Games'][idxa] + 1
#        if df_Matches.iloc[y]['Weather'] == 'rain':
#            df.loc[idxa,'Wins_in_Rain'] = df['Wins_in_Rain'][idxa] + 1
#            df.loc[idxa,'Games_in_Rain'] = df['Games_in_Rain'][idxa] + 1 
#            df.loc[idxh,'Games_in_Rain'] = df['Games_in_Rain'][idxh] + 1 

In [20]:
df.head()

,TeamName,TotalGoals,Wins,Losses,Draws,Total_Games,Wins_in_Rain,Games_in_Rain
0,Aachen,30,6,15,13,34,0,0
1,Arsenal,74,21,10,7,38,0,0
2,Aston Villa,37,7,14,17,38,0,0
3,Augsburg,36,8,12,14,34,0,0
4,Bayern Munich,77,24,7,4,35,0,0


## Create MongoDB Database

In [16]:
client = pymongo.MongoClient('mongodb://localhost:27017')
mod2lab_db = client['mod2lab']
mod2lab_collection = mod2lab_db['mod2lab_collection']

In [21]:
for y in range(len(df)):
    team = {'TeamName': df.iloc[y]['TeamName'],
            'TotalGoals': df.iloc[y]['TotalGoals'],
            'TotalWins': df.iloc[y]['Wins'],
            'HistogramOfWins': plt.hist(df['Wins'], 10),
            'HistogramOfLosses': plt.hist(df['Losses'], 10),
            'Win%DaysRain': round((df.iloc[y]['Wins_in_Rain'])/(df.iloc[y]['Games_in_Rain']), 3)}
    results = mod2lab_collection.insert_one(team)

//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [ ]:
q = mod2lab_collection.find({})
for i in q:
    print(i)

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!